# Enums

## C-Style Enums

In [9]:
#[derive(Debug)]
enum Compass {
    North,
    East,
    South,
    West
}

let direction = Compass::North;

println!("direction = {:?}", direction);

direction = North


* In memory, values of C-style enums are stored as integers. 

In [10]:
use std::mem::size_of;

assert!(size_of::<Compass>() == 1);

### Importing constructors

In [11]:
use self::Compass::*; // self - when enum type is defined in current module; otherwise name of a module

let direction = North;

### Using Integral Values

In [12]:
#[derive(Debug)]
enum HttpStatus {
    Ok = 200,
    NotModified = 304,
    NotFound = 404
}

assert!(size_of::<HttpStatus>() == 2);


### Casting to Integer

In [13]:
let ok_code = HttpStatus::Ok as i32;
assert!(ok_code == 200);

assert!(HttpStatus::NotFound as i32 == 404);

### Casting from Integer

In [14]:
fn http_status_from_u32(status: u32) -> Option<HttpStatus> {
    match status {
        200 => Some(HttpStatus::Ok),
        304 => Some(HttpStatus::NotModified),
        404 => Some(HttpStatus::NotFound),
        _ => None
    }
}

In [15]:
http_status_from_u32(404)

Some(NotFound)

### Deriving From Common Traits

* As with structs, the compiler will implement features like the `==` operator for you, but you have to ask.

In [16]:
#[derive(Copy, Clone, Debug, PartialEq)]
enum TimeUnit {
    Seconds, Minutes, Hours, Days, Months, Years
}

In [17]:
assert!(TimeUnit::Seconds == TimeUnit::Seconds);

### Implementing Methods for Enums

In [18]:
impl TimeUnit {
    /// Return the plural noun for this time unit.
    fn plural(self) -> &'static str {
        match self {
            TimeUnit::Seconds => "seconds",
            TimeUnit::Minutes => "minutes",
            TimeUnit::Hours => "hours",
            TimeUnit::Days => "days",
            TimeUnit::Months => "months",
            TimeUnit::Years => "years"
        }
    }

    /// Return the singular noun for this time unit.
    fn singular(self) -> &'static str {
        self.plural().trim_right_matches('s')
    }
}

In [19]:
let time_unit = TimeUnit::Days;

time_unit.plural()

"days"

## Enums With Data

In [20]:
#[derive(Clone, Debug, PartialEq)]
enum Message {
    Quit, // no data associated
    Move{ x: i32, y: i32 }, // named fields - like a struct
    Write(String), // single String
    ChangeColor(i32, i32, i32) // tuple struct of three values
}

In [21]:
let mut msg = Message::Quit;
println!("msg = {:?}", msg);

msg = Message::Move{ x: 100, y: 200};
println!("msg = {:?}", msg);

msg = Message::Write(String::from("TEXT"));
println!("msg = {:?}", msg);

msg = Message::ChangeColor(255, 0, 255);
println!("msg = {:?}", msg);

msg = Quit
msg = Move { x: 100, y: 200 }
msg = Write("TEXT")
msg = ChangeColor(255, 0, 255)


In [22]:
impl Message {
    fn call(&self) {
        match *self {
            Message::Quit => println!("Quit"),
            Message::Move{ x, y } => println!("Move to x={}, y={}", x, y),
            Message::Write(ref text) => println!("Write '{}'", text),
            Message::ChangeColor(r, g, b) => println!("Change color to r={}, g={}, b={}", r, g, b)
        }
    }
}

In [23]:
msg.call();

Change color to r=255, g=0, b=255


## Generic Enums

### Option

In [ ]:
enum Option<T> {
    None,
    Some(T)
}

### Result

In [ ]:
enum Result<T, E> { 
    Ok(T), 
    Err(E) 
} 

### Real-World Example - BinaryTree

In [25]:
// An ordered collection of `T`s.
enum BinaryTree<T> {
    Empty,
    NonEmpty(Box<TreeNode<T>>)
}

// A part of a BinaryTree.
struct TreeNode<T> {
    element: T,
    left: BinaryTree<T>,
    right: BinaryTree<T>
}

In [26]:
use self::BinaryTree::*;

let jupyter_tree = NonEmpty(Box::new(TreeNode {
    element: "Jupyter",
    left: Empty,
    right: Empty
}));

let mercury_tree = NonEmpty(Box::new(TreeNode {
    element: "Jupyter",
    left: Empty,
    right: Empty
}));

let venus_tree = NonEmpty(Box::new(TreeNode {
    element: "Jupyter",
    left: Empty,
    right: Empty
}));

let mars_tree = NonEmpty(Box::new(TreeNode {
    element: "Saturn",
    left: jupyter_tree,
    right: mercury_tree
}));

let uranus_tree = NonEmpty(Box::new(TreeNode {
    element: "Mars",
    left: Empty,
    right: venus_tree
}));

// the root
let saturn_tree = NonEmpty(Box::new(TreeNode {
    element: "Saturn",
    left: mars_tree,
    right: uranus_tree
}));

## Patterns

In [30]:
enum RoughTime {
    InThePast(TimeUnit, u32),
    JustNow,
    InTheFuture(TimeUnit, u32)
}

In [35]:
fn rough_time_to_english(rt: RoughTime) -> String {
    match rt {
        RoughTime::InThePast(units, 1) => format!("{} ago", units.singular()),
        RoughTime::InThePast(units, count) => format!("{} {} ago", count, units.plural()),
        RoughTime::JustNow => format!("just now"),
        RoughTime::InTheFuture(units, 1) => format!("{} from now", units.singular()),  
        RoughTime::InTheFuture(units, count) => format!("{} {} from now", count, units.plural())  
    }
}

In [38]:
rough_time_to_english(RoughTime::InThePast(TimeUnit::Seconds, 1))

"second ago"

In [39]:
rough_time_to_english(RoughTime::InThePast(TimeUnit::Seconds, 42))

"42 seconds ago"

In [40]:
rough_time_to_english(RoughTime::JustNow)

"just now"

In [41]:
rough_time_to_english(RoughTime::InTheFuture(TimeUnit::Seconds, 42))

"42 seconds from now"

|      Pattern type       |        Example         |                                   Notes                                    |
| ----------------------- | ---------------------- | -------------------------------------------------------------------------- |
| Literal                 | `100`                  | Matches an exact value; the name of a const is also allowed                |
|                         | `"name"`               |                                                                            |
| Range                   | `0 ... 100`            | Matches any value in range, including the end value                        |
|                         | `'a' ... 'k'`          |                                                                            |
| Wildcard                | `_`                    | Matches any value and ignores it                                           |
| Variable                | `name`                 | Like _ but moves or copies the value into a new local variable             |
|                         | `mut count`            |                                                                            |
| ref variable            | `ref field`            | Borrows a reference to the matched value instead of moving or copying it   |
|                         | `ref mut field`        |                                                                            |
| Binding with subpattern | `val @ 0 ... 99`       | Matches the pattern to the right of @, using the variable name to the left |
|                         | `ref circle @`         |                                                                            |
|                         | `Shape::Circle { .. }` |                                                                            |
| Enum pattern	          | `Some(value)`          |                                                                            |
|                         | `None`                 |                                                                            |
|                         | `Message::Ok`          |                                                                            |
| Tuple pattern	          | `(key, value)`         |                                                                            |
|                         | `(r, g, b)`            |                                                                            |
| Struct pattern	      | `Color(r, g, b)`       |                                                                            |
|                         | `Point { x, y }`       |                                                                            |
|                         | `Card { suit: Clubs, rank: n }`  |                                                                  |
|                         | `Account { id, name, .. }`       |                                                                  | 
| Reference	              | `&value`               | Matches only reference values                                              |
|                         | `&(k, v)`	           |                                                                            |
| Multiple patterns	      | `'a' \| 'A'`           | In match only (not valid in let, etc.)                                     |
| Guard expression	      | `x if x * x <= r2`     | In match only (not valid in let, etc.)                                     |